In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile as zf
import re
import random
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
!pip install bert-for-tf2
!pip install sentencepiece

Looking in indexes: https://pypi.doubanio.com/simple
     |████████████████████████████████| 41 kB 515 kB/s eta 0:00:011
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30535 sha256=232da21d8253562ff179985ca2f1e08ae0ff5939d8d79231983dcb3e15c8db96
  Stored in directory: /Users/maywzh/Library/Caches/pip/wheels/af/be/46/5f16520ad3b598fb6dc1b861b89d04d6bd8b4042b82a5263c9
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19471 sha256=c76502f133f5e5d90f65f44c36bcd0a7a19fb1cefcad46591e131ef8d270ecea
  Stored in directory: /Users/maywzh/Library/Caches/pip/wheels/ab/59/7e/9ceb8e939a574beb2a960923c67ef627b04d0aae77361ec6bd
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=7abd3a3c395801f1bd23fd8363f6f9b24f448759b3e7594d43daba832393cda0
  Stored in directory: /Users/maywzh/Library/Caches/pip/wheels/d9/46/c1/44d740bec463fc4f490d639ada6c3e46d802942207814f826f
Successfully built bert-for-tf2

In [3]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
from tensorflow.keras.models import load_model
import bert
from transformers import AutoTokenizer

In [4]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [5]:
train=pd.read_csv("./data/train.csv")
test=pd.read_csv("./data/test.csv")

In [6]:
train.head()

,exercise_text,三角函数,函数奇偶性,导数,平面向量,数列,逻辑与命题关系,集合
0,已知函数其中曲线在点处的切线垂直于轴求的值求函数的极值求导得曲线在点处的切线垂直于轴则函数在...,0,0,1,0,0,0,0
1,已知函数的图像在点处的切线斜率为类猜想等数学思想,0,0,1,0,0,0,0
2,设函数给出如下四个命题若则为奇函数若则函数在上是增函数函数的图象关于点成中心对称图形关于的方...,0,1,0,0,0,1,0
3,若平面向量与的夹角是且则的坐标为试题分析因为所以所以所以考点向量的共线向量的模,0,0,0,1,0,0,0
4,分已知函数是偶函数求的值设若函数与的图象有且只有一个公共点求实数的取值范围当时由题得方程有一...,0,1,0,0,0,0,0


In [9]:
train.columns[1:].tolist()

['三角函数', '函数奇偶性', '导数', '平面向量', '数列', '逻辑与命题关系', '集合']

In [12]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
#FullTokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert_layer = hub.KerasLayer('https://code.aliyun.com/qhduan/zh-bert-wwm/raw/51f499fb7df00cf4d0f283cefd0a7cb34316b866/zh-bert-wwm-L12.tar.gz',output_key='pooled_output',
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

INFO:absl:Downloading TF-Hub Module 'https://code.aliyun.com/qhduan/zh-bert-wwm/raw/51f499fb7df00cf4d0f283cefd0a7cb34316b866/zh-bert-wwm-L12.tar.gz'.
INFO:absl:Downloading https://code.aliyun.com/qhduan/zh-bert-wwm/raw/51f499fb7df00cf4d0f283cefd0a7cb34316b866/zh-bert-wwm-L12.tar.gz: 110.01MB
INFO:absl:Downloading https://code.aliyun.com/qhduan/zh-bert-wwm/raw/51f499fb7df00cf4d0f283cefd0a7cb34316b866/zh-bert-wwm-L12.tar.gz: 220.01MB
INFO:absl:Downloading https://code.aliyun.com/qhduan/zh-bert-wwm/raw/51f499fb7df00cf4d0f283cefd0a7cb34316b866/zh-bert-wwm-L12.tar.gz: 330.01MB
INFO:absl:Downloaded https://code.aliyun.com/qhduan/zh-bert-wwm/raw/51f499fb7df00cf4d0f283cefd0a7cb34316b866/zh-bert-wwm-L12.tar.gz, Total size: 413.52MB
INFO:absl:Downloaded TF-Hub Module 'https://code.aliyun.com/qhduan/zh-bert-wwm/raw/51f499fb7df00cf4d0f283cefd0a7cb34316b866/zh-bert-wwm-L12.tar.gz'.


AttributeError: '_UserObject' object has no attribute 'vocab_file'

In [14]:
X = []
for items in train['exercise_text']:
    X.append(items)

In [15]:
X

['已知函数其中曲线在点处的切线垂直于轴求的值求函数的极值求导得曲线在点处的切线垂直于轴则函数在该点的导数为所以由可得求导得令有或时时时所以时取得极大值时取得极大值考点导数的应用',
 '已知函数的图像在点处的切线斜率为类猜想等数学思想',
 '设函数给出如下四个命题若则为奇函数若则函数在上是增函数函数的图象关于点成中心对称图形关于的方程最多有两个实根其中正确的命题',
 '若平面向量与的夹角是且则的坐标为试题分析因为所以所以所以考点向量的共线向量的模',
 '分已知函数是偶函数求的值设若函数与的图象有且只有一个公共点求实数的取值范围当时由题得方程有一解令可得当时可得恒成立当时可得不成立当时可得不存在当时由题得方程有一解可得不存在或综上所述',
 '已知等比数列中各项都是正数且成等差数列则等于',
 '已知是定义在上的奇函数且当时则的值为',
 '解当时由知考点本试题主要考查了等比数列的定义以及裂项求和的运用点评解决该试题的关键是能利用通项公式与前项和的关系式来得到通项公式同时利用递推关系整体的思想得到同时裂项法得到求和',
 '设全集集合求若集合满足求实数的取值范围',
 '已知函数当时求函数的单调区间当时若恒成立求实数的最小值证明',
 '设是等差数列的前项和若则数列的通项是',
 '在边长为的等边中设析夹角为夹角为夹角为所以考点向量数量积运算点评向量的数量积其中为的夹角本题中向量的夹角的确定是容易出错的地方',
 '已知各项都不相等的等差数列的前项和为且为和的等比中项求数列的通项公式若数列满足且求数列的前项和',
 '已知函数是上的单调减函数且为奇函数则的值',
 '给出如下三个命题四个非零实数依次成等比数列的充要条件是设则若则若则是偶函数其中不正确命题的序号是',
 '设命题命题若是的必要不充分条件求实数的取值范围分',
 '设集合则右图中阴影部分表示的集合为',
 '已知函数求函数的单调区间求证当时对于任意总有成立',
 '在与中间插入个数使这个数成等差数列则这个数列的第项为',
 '设则使函数的定义域为且为奇函数的所有的值为',
 '设是公比为的等比数列令若数列有连续四项在集合中则等于',
 '设数列的各项均为正数若对任意的正整数都有成等差数列且成等比数列求证数列是等差数列如果求数列错误不能通过编辑域代码创建对象的前错误不能通过编辑域代码创建对象项和',

In [18]:
train_data = train["exercise_text"]
test_data = test["exercise_text"]

In [19]:
train_label=train[train.columns[1:].tolist()]

In [20]:
train_label

,三角函数,函数奇偶性,导数,平面向量,数列,逻辑与命题关系,集合
0,0,0,1,0,0,0,0
1,0,0,1,0,0,0,0
2,0,1,0,0,0,1,0
3,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
2972,0,0,0,0,0,1,0
2973,0,0,1,0,0,0,0
2974,0,0,1,0,0,0,0
2975,0,0,0,0,1,1,0


In [22]:
### Import Libraries 
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import seaborn as sns
import pandas as pd
from scipy.sparse import hstack
from nltk.stem import PorterStemmer
import string
!pip install scikit-multilearn             ### For installing scikit-multilearn
#import skmultilearn
from skmultilearn.problem_transform import LabelPowerset        # if error in this line this use import skmultilearn first
word_vec = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 2),
    max_features=20000)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/maywzh/nltk_data'
    - '/Users/maywzh/.pyenv/versions/3.8.6/nltk_data'
    - '/Users/maywzh/.pyenv/versions/3.8.6/share/nltk_data'
    - '/Users/maywzh/.pyenv/versions/3.8.6/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
